<a href="https://colab.research.google.com/github/ShinAsakawa/ShinAsakawa.github.io/blob/master/2022notebooks/2022_0628BERT_SNOW_no_pretrained.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%config InlineBackend.figure_format = 'retina'
import torch
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

from IPython import get_ipython
isColab =  'google.colab' in str(get_ipython())

if isColab:
    !pip install jaconv > /dev/null 2>&1 
    !pip install transformers fugashi ipadic  > /dev/null 2>&1 

In [ ]:
import os
import pandas as pd
import requests
from termcolor import colored
import jaconv

# やさしい日本語をダウンロード
SNOWs={'T15': {'url':"https://filedn.com/lit4DCIlHwxfS1gj9zcYuDJ/SNOW/T15-2020.1.7.xlsx"},
       'T23': {'url':"https://filedn.com/lit4DCIlHwxfS1gj9zcYuDJ/SNOW/T23-2020.1.7.xlsx"},
      }
print('エクセルファイル読込', end='...')
for corpus in SNOWs:
    url = SNOWs[corpus]['url']
    excel_fname = corpus + '-2020.1.7.xlsx'
    
    if not os.path.exists(excel_fname):  # ファイルが存在しない場合ダウンロード
        print(f'url:{url}')
        r = requests.get(url)
        with open(excel_fname, 'wb') as f:
            total_length = int(r.headers.get('content-length'))
            print(f'{excel_fname} をダウンロード中 {total_length} バイト')
            f.write(r.content)

    SNOWs[corpus]['df'] = pd.read_excel(excel_fname)
    SNOWs[corpus]['df'] = SNOWs[corpus]['df'].rename(columns={'#日本語(原文)': 'ja', 
                                                              '#やさしい日本語':'easy_ja',
                                                              '#英語(原文)':'en'})
# 2 つのデータをあわせる    
_snow = SNOWs['T15']['df']['easy_ja'].tolist() + SNOWs['T23']['df']['easy_ja'].tolist()
snow = [jaconv.normalize(line, 'NFKC') for line in _snow] # 正規化

エクセルファイル読込...url:https://filedn.com/lit4DCIlHwxfS1gj9zcYuDJ/SNOW/T15-2020.1.7.xlsx
T15-2020.1.7.xlsx をダウンロード中 3634132 バイト
url:https://filedn.com/lit4DCIlHwxfS1gj9zcYuDJ/SNOW/T23-2020.1.7.xlsx
T23-2020.1.7.xlsx をダウンロード中 3641507 バイト


BERT の事前訓練には，マスク化言語モデル (**MLM**) と次文予測 (**NSP**) という 2 つの独自の学習アプローチがある。
ここでは，マスク化言語モデルを用いて，モデルを微調整 fine-tuning する方法を試してみることとする。

# 1. マスク化言語モデル MLM

MLM とは BERT モデルに対して，文を入力として与え，BERT 内部の重みを最適化して，側に同じ文を出力することである。
このとき，文中の任意の単語，または単語の断片を マスクトークンに置き換えることを行う。
BERT はこの，マスクトークンの位置に正しいトークンを予測することが求められる。

実際に BERT にその入力文を与える前に，トークンを定義するなどが，必要になる。

<center>
<img src="https://miro.medium.com/max/1400/1*phTLnQ8itb3ZX5_h9BWjWw.png" width="600px"><br/>
本図では，トークン を BERT に渡す前に，リンカーン・トークンを [MASK] に置き換えてマスクしている。
</center>

すなわち，実際には不完全な文章を入力して，BERT に，その文章を完成させるように依頼している。
国語の教科教育，あるいは外国語の習得おける，穴埋め問題とみなしうる。

## 1.1 マスクを埋める

例えば以下のような文章が与えられたとする:

`秋 に は ， ___ が 木 から 落ちる 。`

アンダーライン部分の単語を推定することを考えてみる。
答えは，所与の文章かから，文脈を類推し予測していることになる。

「落ちる」 と 「木」 という単語が出てきましたが，足りない単語は木から落ちるものだということがわかる。

どんぐり，枝，葉など，木から落ちるものはたくさんある。
だが，秋という別の条件があるので，秋に木から落ちる可能性が最も高いのは葉だということで，検索対象が絞られる。

人間はは，一般的な世界の知識と言語的な理解を組み合わせて，その結論を導き出す。
BERT の場合，この推測は，コーパスから，たくさんの文章を与えられることで，言語パターンを学んでいることから適切な単語を得ることとなる。

BERT は，秋，木，葉が何であるかを知らないかもしれません。
だが，言語パターンとこれらの単語の文脈から，答えが葉である可能性が最も高いことを知ることとなる。

この処理の結果，BERT にとっては，使用されている言語のスタイルの理解度が向上する。


## 1.2 実際の処理

MLM が何をしているかは理解できたと思うが，実際にはどのように機能するのだろうか?
コード上で必要となる論理的なステップは何だろうか？
以下のように考えることができよう:

1. テキストをトークン化する
通常の変換機と同じように，まずテキストをトークン化する。
BERT の標準的な手続きでは，トークン化により，3 つの異なるテンソルが得られる。

* input_ids
* token_type_ids
* attention_mask

MLM には `token_type_ids` は必要ない。
本例では `attention_mask` はそれほど重要ではない。

この問題設定では `input_ids` テンソルが重要である。
`input_ids` はトークン化済みの文表現であり，これを修正していくこととする。

2. `labels tensor` を作成

ここではモデルを訓練しているので，損失を計算して最適化するためのラベルテンソルが必要である。
実際には `labels tensor` は単純に `input_ids` なので，これをコピーするだけである。

3. `input_ids` のトークンをマスクする

label 用の `input_ids` のコピーを作成した後，トークン系列内のランダムな位置のトークンを選択し，マスクする。


BERT 論文では，モデルの事前訓練中に，いくつかの追加ルールを用いて，各トークンを 15％ の確率でマスク化している。
ここではこれを簡略化して，各単語を 15％ の確率でマスク化することとする。


4. 損失を計算する

`input_ids` と `labels` のテンソルを BERT モデルで処理し，両者の間の損失を計算する。
この損失値を用いて，BERT による必要な勾配変化を計算し，モデルの重みを最適化する。

<center>
<img src="https://miro.medium.com/max/1400/1*0KvOrY6rY055m9oq36HRkg.png" width="600px"><br/>
<div style="text-align:left; width:66%; background-color:cornsilk">

512 個のトークンはすべて，モデルの語彙サイズに等しいベクトル長を持つ。
このベクトルを BERT に与えることにより，最終的な出力埋め込みベクトルであるロジット(確率比) が生成される。
予測されたトークン ID は，ソフトマックスと argmax 変換を用いて，このロジットから抽出される。
</div>    
</center>    

損失は，各「トークン」の出力確率分布と，真のワンホット符号化ラベルとの差として計算される。

# 2. マスク化言語モデル MLM の実装

HuggingFace のトランスフォーマーと PyTorch を用いて実装を検討する。
標的言語が英語の場合には `bert-base-uncased` モデルを使用する。
日本語の場合には，複数の訓練済モデルが提案されている。
ここでは，東北大学版の訓練済モデル `cl-tohoku/bert-base-japanese` を用いた実装を試みる。

まず全てをインポートして初期化する。

In [ ]:
import torch
import transformers    
from transformers import BertJapaneseTokenizer
from transformers import BertForMaskedLM
from transformers import BertConfig

# BERT 訓練済モデルをダウロード
model_name_ja = 'cl-tohoku/bert-base-japanese'  # 東北大学乾研による 日本語 BERT 実装
# see https://huggingface.co/sonoisa/sentence-bert-base-ja-mean-tokens-v2
# model_name_ja = 'sonoisa/sentence-bert-base-ja-mean-tokens-v2'  # 東北大学乾研による 日本語 BERT 実装

tknz = BertJapaneseTokenizer.from_pretrained(model_name_ja)
# ここだけが異なる
config = BertConfig.from_pretrained(model_name_ja)  
bert_model = BertForMaskedLM(config=config) #return_dict = True)
# bert_model = BertForMaskedLM.from_pretrained(model_name_ja, return_dict = True)

# リソースの選択（CPU/GPU）
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

最大トークン長 `max_length` を設定する必要があるため，SNOW コーパスの最大長を求めておく。
BPE (Byte-per-encoding) では，任意の文章のトークン数が，その文章の文字数以上になることはあり得ないことから，
SNOW コーパスに現れる全文から，最大文字数の文章を探して，その文章の文字数を最大値として利用することとしてみる。

BPE の詳細については，ここでは触れない。
簡単に説明すると，どのような言語でも，文字数 <= 単語数 という関係が成り立つ。
そのため，入力ベクトル表現の次元数は，文字ベースではれば小さくなり，単語ベースであれば，総語彙分の次元を用意しなければならない。
一方，単語ベースでは任意の単語の持つ意味表現を，表現できてはいない。
そこで，単語ベースの次元数の少なさと，単語ベースの意味表現の長所の折衷案を考える。
まず単語ベースの表現を考えて全文を走査する。
そして，頻出する文字の並びを，新たなトークンとみなして，新たなトークンを作成する。
望むトークン数の上限に達するまで，この操作を繰り返す。
これにより，単語ベースの入力ベクトル次元数の小ささと，
頻出語彙は登録されているため，意味情報を反映できるという，両者の長所を捉えた表現が可能である。
これが BPE の本質である。

In [ ]:
max_length = 0
for line in snow:
    max_length = len(line) if len(line) > max_length else max_length
print(f'やさしい日本語における一文の最長文字数:{max_length}')
inputs = tknz(snow, return_tensors='pt', max_length=max_length+1, truncation=True, padding='max_length')
# トークンの分割数が文字数以上になることはないので `max_length=max_length+1`とした
print(inputs.input_ids.shape)

やさしい日本語における一文の最長文字数:99
torch.Size([84300, 100])


In [ ]:
#type(tknz.vocab)  # collections.OrderedDict
#len(tknz.vocab)    # 3200

# BERT のトークン化器に登録されている語彙のうち，最後の 30 トークンを表示させてみる
print(list(tknz.vocab.keys())[-30:])

# やさしい日本語コーパスの最初の文のトークン化を表示させてみる
print(tknz.tokenize(snow[0]))

# 直上の，やさしい日本語コーパスの最初の文に対応するトークン ID を表示させてみる
print(tknz(snow[0])['input_ids'])

['##鵠', '##Š', '##ˈ', '##ヱ', '##丑', '##唖', '##娩', '##彪', '##忽', '##托', '##桧', '##楡', '##珀', '##祢', '##稷', '##竈', '##粥', '##臧', '##蟄', '##訣', '##賽', '##銑', '##鯱', '##ē', '##ł', '##з', '##侃', '##嬪', '##崋', '##巽']
['誰', 'が', '一番', 'に', '着く', 'か', '私', 'に', 'は', '分かり', 'ませ', 'ん', '。']
[2, 3654, 14, 4749, 7, 17324, 29, 1325, 7, 9, 14604, 6769, 1058, 8, 3]


## 2.1 文章のトークン化

試みに，最初の 3 文だけトークン化器にとおして，戻り値を観察して理解を深めておきたい。

In [ ]:
print(snow[:3])  # やさしい日本語コーパスの最初の 3 文

# やさしい日本語コーパスの最初の 3 文をトークナイズして `inputs` に代入
inputs = tknz(snow[:3], return_tensors='pt', padding=True)

# 代入した結果が `dict` として返ってくるので，その `dict` の key を表示
print(inputs.keys())

# 返ってきた `dict` の内容を表示
print(inputs)

['誰が一番に着くか私には分かりません。', '多くの動物が人間によって殺された。', '私はテニス部員です。']
dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
{'input_ids': tensor([[    2,  3654,    14,  4749,     7, 17324,    29,  1325,     7,     9,
         14604,  6769,  1058,     8,     3],
        [    2,   378,     5,  2056,    14,  1410,   230,  6030,    20,    10,
             8,     3,     0,     0,     0],
        [    2,  1325,     9,  6889, 12328,  2992,     8,     3,     0,     0,
             0,     0,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]])}


## ラベル作成

`input_ids` テンソルを新しい labels テンソルに複製 `clone()` して，結果を `inputs` 変数に格納する。


In [ ]:
inputs['labels'] = inputs.input_ids.detach().clone()
print(inputs)

{'input_ids': tensor([[    2,  3654,    14,  4749,     7, 17324,    29,  1325,     7,     9,
         14604,  6769,  1058,     8,     3],
        [    2,   378,     5,  2056,    14,  1410,   230,  6030,    20,    10,
             8,     3,     0,     0,     0],
        [    2,  1325,     9,  6889, 12328,  2992,     8,     3,     0,     0,
             0,     0,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]]), 'labels': tensor([[    2,  3654,    14,  4749,     7, 17324,    29,  1325,     7,     9,
         14604,  6769,  1058,     8,     3],
        [    2,   378,     5,  2056,    14,  1410,   230,  6030,    20,    10,
             8,     3,     0,

## 2.2 マスクの作成

BERT の原著論文では，マスク化率が 15% のときに，BERT の性能が高くなるとの記述がある。
そこで，この 15% の割合で，ランダムなマスクを作成する。

トークンを 15% の確率でマスク化するためには `torch.rand` を用いて乱数を発生させ，その発生した値と $<0.15$ を比較することで，マスク化すべきトークン位置を決定することとする。
このときの，マスクを示す配列を `mask_arr` とする。

In [ ]:
# input_idsと同じ次元の浮動小数点数のランダムな配列を作成
rand = torch.rand(inputs.input_ids.shape)

# 乱数配列が 0.15 より小さい場合は `true` を設定
mask_arr = rand < 0.15

# 結果としてできあがったマスク化配列を印字
print(mask_arr) 

tensor([[False, False, False, False, False, False, False, False, False, False,
         False,  True, False, False, False],
        [False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False],
        [False, False, False,  True,  True, False, False, False, False,  True,
         False, False,  True, False, False]])


- [MASK] トークンを配置する場所を選ぶために mask_arr を用いる。
- 駄菓子菓子，[CLS] トークンや [SEP] トークンなどの特殊トークン (それぞれ `tknz.cls_token_id` と `tknz.sep_token_id` で取得可能) の上に MASK トークンを配置したくはない。

そこで，さらに条件を追加する必要がある。
トークン ID `tknz.cls_token_id` または `tknz.sep_token_id` を含む位置をチェックしてみることとする。

その前に，どの特殊トークンが，どのトークン ID を持つのかを調べてみよう。

In [ ]:
print(tknz.vocab['[MASK]'], '=', tknz.mask_token_id)
print(tknz.vocab['[CLS]'], '=', tknz.cls_token_id)
print(tknz.vocab['[SEP]'], '=', tknz.sep_token_id)

4 = 4
2 = 2
3 = 3


上記のように，マスクトークンは `.mask_token_id`, 文トークンは `.cls_token_id`, 2 文の分離トークンは `sep_token_id` で参照される。
これら特殊トークンの語彙辞書内のトークン番号は， `.vocab[トークン]` で参照可能である。

In [ ]:
# 特殊トークンを表示
print(tknz.special_tokens_map)

# 各特殊トークンが，どのようなトークン番号を持つかを表示
print([(value,tknz.vocab[value]) for token, value in tknz.special_tokens_map.items()])

{'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}
[('[UNK]', 1), ('[SEP]', 3), ('[PAD]', 0), ('[CLS]', 2), ('[MASK]', 4)]


In [ ]:
# 入力文 `input_ids` 内のトークンが [CLS] ではなく，かつ，分離トークン [SEP] でもない 場合には `True` となる
print((inputs.input_ids != tknz.cls_token_id) * (inputs.input_ids != tknz.sep_token_id))

tensor([[False,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True, False],
        [False,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True, False,  True,  True,  True],
        [False,  True,  True,  True,  True,  True,  True, False,  True,  True,
          True,  True,  True,  True,  True]])


In [ ]:
# 入力文 `input_ids` 内のトークンが [CLS] ではなく，かつ，分離トークン [SEP] でもなく
# かつ，15% のマスク配列が `True` である場合には `True` となる
mask_arr = (rand < 0.15) * (inputs.input_ids != tknz.cls_token_id) * (inputs.input_ids != tknz.sep_token_id)
print(mask_arr)

tensor([[False, False, False, False, False, False, False, False, False, False,
         False,  True, False, False, False],
        [False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False],
        [False, False, False,  True,  True, False, False, False, False,  True,
         False, False,  True, False, False]])


In [ ]:
# 直上セルで作成した mask_arr から selection を計算して作成
selection = torch.flatten((mask_arr).nonzero()).tolist()
print(selection)

[0, 11, 2, 3, 2, 4, 2, 9, 2, 12]


In [ ]:
# selection で表現された文中の位置情報を `inputs.input_ids` へ適用し MASK トークンに置き換える
inputs.input_ids[0, selection] = tknz.vocab[tknz.mask_token]
print(inputs)

{'input_ids': tensor([[    4,  3654,     4,     4,     4, 17324,    29,  1325,     7,     4,
         14604,     4,     4,     8,     3],
        [    2,   378,     5,  2056,    14,  1410,   230,  6030,    20,    10,
             8,     3,     0,     0,     0],
        [    2,  1325,     9,  6889, 12328,  2992,     8,     3,     0,     0,
             0,     0,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]]), 'labels': tensor([[    2,  3654,    14,  4749,     7, 17324,    29,  1325,     7,     9,
         14604,  6769,  1058,     8,     3],
        [    2,   378,     5,  2056,    14,  1410,   230,  6030,    20,    10,
             8,     3,     0,

上の結果から `input_ids` テンソル内に MASK トークン `4` が存在することを確認せよ。

## 2.3 損失の計算

最後のステップは，一般的なモデルの訓練処理と変わりない。
`input_ids` テンソルと `labels` テンソルが `input` に入っているので，これをモデルに渡してモデルの損失を返すことができる。

In [ ]:
outputs = bert_model(**inputs)

In [ ]:
# 戻り値 `outputs` は `dict` であり，この `output` 辞書のキーには `loss` (損失値) と `logits` (確率) とがある。
print(outputs.keys())

odict_keys(['loss', 'logits'])


In [ ]:
# 損失値を印字
print(outputs.loss)

tensor(10.4411, grad_fn=<NllLossBackward0>)


# 3. 学習


In [ ]:
%%time
# やさしい日本語コーパス SNOW 全文をトークン化器に与え，結果を `inputs` に代入する
inputs = tknz(snow, return_tensors='pt', max_length=100, truncation=True, padding='max_length')

# 結果を表示
print(inputs)

{'input_ids': tensor([[    2,  3654,    14,  ...,     0,     0,     0],
        [    2,   378,     5,  ...,     0,     0,     0],
        [    2,  1325,     9,  ...,     0,     0,     0],
        ...,
        [    2, 16682,    11,  ...,     0,     0,     0],
        [    2,  7843,     7,  ...,     0,     0,     0],
        [    2,   811,     5,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}
CPU times: user 11.4 s, sys: 189 ms, total: 11.6 s
Wall time: 11.6 s


In [ ]:
print(f"inputs['input_ids'].size() 戻り値 inputs の大きさ:{inputs['input_ids'].size()}")
print(f'type(inputs.input_ids.detach()) 戻り値 inputs の型:{type(inputs.input_ids.detach())}')
print(f'type(inputs.input_ids.detach().numpy()) 戻り値を numpy 行列に変換した際の型:{type(inputs.input_ids.detach().numpy())}')
print(f'inputs.input_ids.detach().numpy().shape 戻り値を numpy 行列に変換した際の行列サイズ:{inputs.input_ids.detach().numpy().shape}')
print(f'inputs.input_ids.detach().numpy()[:2] numpy 変換した際の最初の 2 文:{inputs.input_ids.detach().numpy()[:2]}')
print(f'type(snow) snow の型:{type(snow)}, len(snow) snow のデータ長:{len(snow)}')

inputs['input_ids'].size() 戻り値 inputs の大きさ:torch.Size([84300, 100])
type(inputs.input_ids.detach()) 戻り値 inputs の型:<class 'torch.Tensor'>
type(inputs.input_ids.detach().numpy()) 戻り値を numpy 行列に変換した際の型:<class 'numpy.ndarray'>
inputs.input_ids.detach().numpy().shape 戻り値を numpy 行列に変換した際の行列サイズ:(84300, 100)
inputs.input_ids.detach().numpy()[:2] numpy 変換した際の最初の 2 文:[[    2  3654    14  4749     7 17324    29  1325     7     9 14604  6769
   1058     8     3     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0]
 [    2   378     5  

In [ ]:
# 今一度，各トークン番号が，どのようなトークンを表しているのかを
# トークナイザの `convert_ids_to_tokens()` 関数を使ってトークンに再変換して印字
for line in inputs.input_ids[:2].detach().numpy():
    print(tknz.convert_ids_to_tokens(line))

['[CLS]', '誰', 'が', '一番', 'に', '着く', 'か', '私', 'に', 'は', '分かり', 'ませ', 'ん', '。', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
['[CLS]', '多く', 'の', '動物', 'が', '人間', 'によって', '殺さ', 'れ', 'た', '。', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]'

次に `input_id` をクローンして，ラベルテンソルを作成します。


In [ ]:
inputs['labels'] = inputs.input_ids.detach().clone()
print(inputs.keys())


dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])


次に，マスクのコードですが，マスクに PAD トークンを含めてはいけない (CLS や SEP では以前のとおりにあつかう)。

In [ ]:
print(f'tknz.pad_token_id:{tknz.pad_token_id}')
print(f'tknz.mask_token_id:{tknz.mask_token_id}')
print(f'tknz.sep_token_id:{tknz.sep_token_id}')

tknz.pad_token_id:0
tknz.mask_token_id:4
tknz.sep_token_id:3


In [ ]:
# input_ids テンソルと同じ次元の乱数配列を作成 
rand = torch.rand(inputs.input_ids.shape)

# mask 配列の作成 
mask_arr = (rand < 0.15) * (inputs.input_ids != tknz.cls_token_id) * \
           (inputs.input_ids != tknz.sep_token_id) * (inputs.input_ids != tknz.pad_token_id)
print(mask_arr)

tensor([[False,  True, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        ...,
        [False, False,  True,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False]])


In [ ]:
selection = []

for i in range(inputs.input_ids.shape[0]):
    selection.append(
        torch.flatten(mask_arr[i].nonzero()).tolist()
    )
print(selection[:5])


[[1], [9], [6], [6], [5]]


次に，これらのインデックスを `input_ids` の各行に適用し，これらのインデックスの値をそれぞれ `tknz.mask_token_id` として割り当てる。


In [ ]:
for i in range(inputs.input_ids.shape[0]):
    inputs.input_ids[i, selection[i]] = tknz.mask_token_id

print(tknz.mask_token_id)
print(inputs.input_ids[:3])


4
tensor([[    2,     4,    14,  4749,     7, 17324,    29,  1325,     7,     9,
         14604,  6769,  1058,     8,     3,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
        [    2,   378,     5,  2056,    14,  1410,   230,  6030,    20,     4,
             8,     3,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0

`mask_arr` テンソルの `True` 値と同じ位置に `tknz.mask_token_id` が割り当てられている

これで入力テンソルの準備が整い，学習時にモデルに入力するための設定を始めることができる。

学習時には PyTorch の `DataLoader` を使ってデータを読み込みます。
これを使うには，データを PyTorch の `Dataset` オブジェクトにフォーマットする必要がある


In [ ]:
class snowDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
        
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    
    def __len__(self):
        return len(self.encodings.input_ids)
    
dataset = snowDataset(inputs) 

`dataloader` を初期化する。
`dataloader` は，訓練時にモデルにデータを読み込むために使用。


In [ ]:
loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

これで，反復訓練に入る準備が整った。
反復学習を始める前に，以下の 3 つを設定する必要がある:

1. モデルを GPU/CPU (GPU が利用可能あれば) に移動
2. モデルの訓練モードを有効にする
3. 重み付けされた重み崩壊付き最適化 `AdamW` を初期化


In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
bert_model.to(device) # モデルを選択したデバイスに移動
bert_model.train();   # 訓練モードに設定

In [ ]:
from torch.optim import AdamW
optim = AdamW(bert_model.parameters(), lr=5e-5)  # 最適化関数を初期化

これでようやくセットアップが完了し，訓練を開始することができる。
ここでは PyTorch の典型的な訓練ループを導入してみる。


In [ ]:
%%time
from collections import OrderedDict
from tqdm.notebook import tqdm
#from tqdm import tqdm  # for our progress bar

epochs = 2
for epoch in range(epochs):

    loop = tqdm(loader, leave=True)
    for batch in loop:
        
        optim.zero_grad()
        
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = bert_model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optim.step()
        
        loop.set_description(f'エポック {epoch}')
        loop.set_postfix(OrderedDict(loss=loss.item()))

  0%|          | 0/5269 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


  0%|          | 0/5269 [00:00<?, ?it/s]

CPU times: user 22min 31s, sys: 13min 37s, total: 36min 8s
Wall time: 35min 51s


In [ ]:
# 学習結果を保存
torch.save(bert_model,'2022_0628bert_model_ja_no_pretrained.pt')

In [ ]:
if isColab:
    # 保存した結果をダウンロード
    from google.colab import files
    files.download('2022_0628bert_model_ja_no_pretrained.pt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#!ls -tl "drive/MyDrive/Colab Notebooks"
#!mkdir drive/MyDrive/2022torch_pt
!cp -p 2022_0628bert_model_ja_no_pretrained.pt drive/MyDrive/2022torch_pt

In [ ]:
class snowDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

dataset = snowDataset(inputs)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

for data in dataloader:
    print(data.keys(), type(data))
    break